In [7]:
import os
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Initialize Flask application
app = Flask(__name__, static_url_path='/static')

# Load the skin cancer classifier model
model = load_model('models/imageclassifiers.h5')
allowed_classes = ['basal cell carcinoma', 'melanoma', 'squamous cell carcinoma']

# Define allowed file extensions
ALLOWED_EXTENSIONS = {'jpg', 'jpeg', 'png'}

# Configure upload folder
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


def allowed_file(filename):
    # Check if the file has an allowed extension
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


def preprocess_image(image):
    # Preprocess the image for prediction
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))  # Adjust the size according to the model requirements
    image = image.astype('float32') / 255.0
    return image


def classify_image(image):
    # Classify the image using the loaded model
    preprocessed_image = preprocess_image(image)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
    prediction = model.predict(preprocessed_image)
    predicted_label_index = np.argmax(prediction)
    predicted_label = allowed_classes[predicted_label_index]
    
    return predicted_label if predicted_label in allowed_classes else 'Error: Image does not belong to a skin cancer class.'


@app.route('/', methods=['GET'])
def index():
    # Render the index.html template
    return render_template('index.html')



@app.route('/predict', methods=['GET','POST'])
def upload():
    # Handle file upload from the front-end
    if 'file' not in request.files:
        return 'No file uploaded', 400

    file = request.files['file']

    if file.filename == '':
        return 'No selected file', 400

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Load and classify the uploaded image
        image = cv2.imread(filepath)
        predicted_label = classify_image(image)

        # Remove the uploaded file
        os.remove(filepath)

        return f'{predicted_label}'

    return 'Invalid file', 400


if __name__ == '__main__':
    app.run()

%tb


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
